In [ ]:
import numpy as np

In [6]:
import gnupg
import keyring

In [7]:
gpg = gnupg.GPG()
with open('C:\\Users\\Administrador\\Repositorios\\binance_trader\\.env.gpg', 'rb') as file:
    datos = gpg.decrypt_file(file, passphrase=keyring.get_password("GPG_Passphrase", "gpg_python"))
if datos.ok:
    env_vars = dict(line.decode('utf-8').split('=', 1) for line in datos.data.splitlines())
    api_key = env_vars.get('API_KEY')
    secret_key = env_vars.get('SECRET_KEY')